##  Five Day Weather Forecast Information

In [1]:
import requests
import pandas as pd
import numpy as np
import datetime
import json

In [2]:
weather_forecast = requests.get("https://pro.openweathermap.org/data/2.5/forecast/hourly?q=London&appid=9a8ddf0c4ecf94975ce6a1c702afbe2d")
fiveDayJSON = weather_forecast.json()
fiveDayJSON

{'cod': '200',
 'message': 0,
 'cnt': 96,
 'list': [{'dt': 1625050800,
   'main': {'temp': 288.22,
    'feels_like': 287.85,
    'temp_min': 288.22,
    'temp_max': 288.24,
    'pressure': 1017,
    'sea_level': 1017,
    'grnd_level': 1014,
    'humidity': 79,
    'temp_kf': -0.02},
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04d'}],
   'clouds': {'all': 92},
   'wind': {'speed': 3.57, 'deg': 344, 'gust': 4.95},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2021-06-30 11:00:00'},
  {'dt': 1625054400,
   'main': {'temp': 288.5,
    'feels_like': 288.08,
    'temp_min': 288.5,
    'temp_max': 288.94,
    'pressure': 1017,
    'sea_level': 1017,
    'grnd_level': 1014,
    'humidity': 76,
    'temp_kf': -0.44},
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04d'}],
   'clouds': {'all': 94},
   'wind': {'speed': 3.38, 'deg': 343, 'gust': 4.76},
  

## Normalise the JSON file

In [3]:
fiveDayForecast = pd.json_normalize(fiveDayJSON, record_path =['list'])
fiveDayForecast.head()

,dt,weather,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.sea_level,main.grnd_level,main.humidity,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,sys.pod,rain.1h
0,1625050800,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000,0.0,2021-06-30 11:00:00,288.22,287.85,288.22,288.24,1017,1017,1014,79,-0.02,92,3.57,344,4.95,d,NaN
1,1625054400,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000,0.0,2021-06-30 12:00:00,288.50,288.08,288.50,288.94,1017,1017,1014,76,-0.44,94,3.38,343,4.76,d,NaN
2,1625058000,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000,0.0,2021-06-30 13:00:00,289.10,288.63,289.10,289.70,1017,1017,1014,72,-0.60,96,3.65,340,4.89,d,NaN
3,1625061600,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000,0.0,2021-06-30 14:00:00,289.70,289.19,289.70,290.07,1017,1017,1014,68,-0.37,98,3.71,341,4.90,d,NaN
4,1625065200,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000,0.0,2021-06-30 15:00:00,290.41,289.84,290.41,290.41,1017,1017,1014,63,0.00,100,4.04,339,5.23,d,NaN


In [4]:
forcast_type = fiveDayForecast[['main.temp','wind.speed', 'main.humidity', 'dt']]
forcast_type.head()

,main.temp,wind.speed,main.humidity,dt
0,288.22,3.57,79,1625050800
1,288.50,3.38,76,1625054400
2,289.10,3.65,72,1625058000
3,289.70,3.71,68,1625061600
4,290.41,4.04,63,1625065200


In [5]:
forcast_fiveday = pd.json_normalize(fiveDayJSON, record_path = ['list', 'weather'])[['main','description']]
forcast_fiveday.head()

,main,description
0,Clouds,overcast clouds
1,Clouds,overcast clouds
2,Clouds,overcast clouds
3,Clouds,overcast clouds
4,Clouds,overcast clouds


## Checking whether Rain is Forecast

In [6]:
forecast_list = fiveDayJSON['list']
rain = []

for i in range(len(forecast_list)):
    if 'rain' in forecast_list[i]:
        rain.append(forecast_list[i]['rain'].pop('1h'))
    else:
        rain.append(0)  
df_rain = pd.DataFrame(rain, columns=["Rain"])


## Checking whether Snow is Forecast

In [7]:
forecast_list = fiveDayJSON['list']
snow = []

for i in range(len(forecast_list)):
    if 'snow' in forecast_list[i]:
        snow.append(forecast_list[i]['snow'].pop('1h'))
    else:
        snow.append(0)  
df_snow = pd.DataFrame(snow, columns=["Snow"])

In [8]:
fiveDayForecast = pd.concat([forcast_fiveday, df_rain, forcast_type, df_snow], axis=1)
fiveDayForecast["main.temp"] = fiveDayForecast["main.temp"]-273.15
fiveDayForecast.head()

,main,description,Rain,main.temp,wind.speed,main.humidity,dt,Snow
0,Clouds,overcast clouds,0.0,15.07,3.57,79,1625050800,0
1,Clouds,overcast clouds,0.0,15.35,3.38,76,1625054400,0
2,Clouds,overcast clouds,0.0,15.95,3.65,72,1625058000,0
3,Clouds,overcast clouds,0.0,16.55,3.71,68,1625061600,0
4,Clouds,overcast clouds,0.0,17.26,4.04,63,1625065200,0


In [9]:
fiveDayForecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   main           96 non-null     object 
 1   description    96 non-null     object 
 2   Rain           96 non-null     float64
 3   main.temp      96 non-null     float64
 4   wind.speed     96 non-null     float64
 5   main.humidity  96 non-null     int64  
 6   dt             96 non-null     int64  
 7   Snow           96 non-null     int64  
dtypes: float64(3), int64(3), object(2)
memory usage: 6.1+ KB


## Finalised Dataferame

In [10]:
fiveDayForecast['dt'] = pd.to_datetime(fiveDayForecast['dt'], unit='s')
fiveDayForecast = fiveDayForecast.rename(columns={"main.temp": "temp", "wind.speed": "wind_speed", "main.humidity": "humidity"})
fiveDayForecast

,main,description,Rain,temp,wind_speed,humidity,dt,Snow
0,Clouds,overcast clouds,0.00,15.07,3.57,79,2021-06-30 11:00:00,0
1,Clouds,overcast clouds,0.00,15.35,3.38,76,2021-06-30 12:00:00,0
2,Clouds,overcast clouds,0.00,15.95,3.65,72,2021-06-30 13:00:00,0
3,Clouds,overcast clouds,0.00,16.55,3.71,68,2021-06-30 14:00:00,0
4,Clouds,overcast clouds,0.00,17.26,4.04,63,2021-06-30 15:00:00,0
...,...,...,...,...,...,...,...,...
91,Clouds,overcast clouds,0.00,15.29,1.26,88,2021-07-04 06:00:00,0
92,Clouds,overcast clouds,0.00,16.54,1.34,81,2021-07-04 07:00:00,0
93,Clouds,overcast clouds,0.00,18.13,1.53,71,2021-07-04 08:00:00,0
94,Clouds,overcast clouds,0.00,19.61,1.85,64,2021-07-04 09:00:00,0


In [12]:
fiveDayForecast.to_csv("fiveDayForecast.csv", 
                                         index_label='Feature')